# 1. Importation des librairies
--------------------------

In [4550]:
from math import sin, cos, sqrt, atan2,radians
import pandas as pd
import numpy as np

# 2. Définition du chemin de lecture et lecture des fichiers
---------------------------------------------------

In [4551]:
PATH = '~/project9/V2/dataset/'

coordinates_df = pd.read_csv(PATH + 'countries_codes_and_coordinates.csv', sep=',', skipinitialspace=True)
food_availability_df = pd.read_csv(PATH + 'dispo_alim.csv')
eco_df = pd.read_csv(PATH + 'eco_datas.csv')
fao_codes_df = pd.read_csv(PATH + 'fao_code_def.csv')
languages_df = pd.read_csv(PATH + 'languages.csv')
political_stability_df = pd.read_csv(PATH + 'political_stability.csv')
population_df = pd.read_csv(PATH + 'population.csv')


Dans les différents tableau tableau nous avons decidé de garder `les variables suivantes` : <br>
<li>Stabilité politique</li>
<li>PIB par habitant (composite de la table PIB et population)</li>
<li>Taux d'imporation (importation / Disponibilité intérieur)</li>
<li>Production</li>
<li>Langue la plus parlé dans le pays</li>
<li>Distance de la France au pays cible</li>

# 3. Mise des différentes tables aux mêmes normes
-------------------------------------------------------------
## 3.1 Coordonnées GPS
--------------------------

In [4552]:
coordinates_df.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [4553]:
coordinates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              244 non-null    object 
 1   Alpha-2 code         243 non-null    object 
 2   Alpha-3 code         244 non-null    object 
 3   Numeric code         244 non-null    int64  
 4   Latitude (average)   244 non-null    float64
 5   Longitude (average)  244 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 11.6+ KB


Nous allons `traduire` le nom des pays de `la même manière que la FAO`.

In [4554]:
fao_codes_df.head()

,Code Pays,Pays,Code M49,Code ISO2,Code ISO3,Année de début,Año de inicio
0,2,Afghanistan,4.0,AF,AFG,NaN,NaN
1,5100,Afrique,2.0,F5100,X06,NaN,NaN
2,5104,Afrique australe,18.0,F5104,F5104,NaN,NaN
3,5102,Afrique centrale,17.0,F5102,F5102,NaN,NaN
4,429,Afrique du Nord (Soudan Excl),746.0,F429,F429,NaN,NaN


In [4555]:
fao_codes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Code Pays       341 non-null    int64  
 1   Pays            341 non-null    object 
 2   Code M49        341 non-null    float64
 3   Code ISO2       340 non-null    object 
 4   Code ISO3       341 non-null    object 
 5   Année de début  39 non-null     float64
 6   Año de inicio   9 non-null      float64
dtypes: float64(3), int64(1), object(3)
memory usage: 18.8+ KB


On s'aperçoit que le nom de la colonnes ISO3 n'est pas les même entre les différent tableau. Changeons-les.

In [4556]:
features = coordinates_df.columns
coordinates_df = coordinates_df.rename(columns={features[2] : 'Code ISO3'})
features = coordinates_df.columns
coordinates_df.head()

,Country,Alpha-2 code,Code ISO3,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


Nous pouvons maintenant `traduire` le nom des pays.

In [4557]:
coordinates_df = pd.merge(fao_codes_df[['Pays',features[2]]],coordinates_df,how='right',on = features[2])

coordinates_df

,Pays,Code ISO3,Country,Alpha-2 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AFG,Afghanistan,AF,4,33.0000,65.0
1,Albanie,ALB,Albania,AL,8,41.0000,20.0
2,Algérie,DZA,Algeria,DZ,12,28.0000,3.0
3,Samoa américaines,ASM,American Samoa,AS,16,-14.3333,-170.0
4,Andorre,AND,Andorra,AD,20,42.5000,1.6
...,...,...,...,...,...,...,...
239,Îles Wallis-et-Futuna,WLF,Wallis and Futuna,WF,876,-13.3000,-176.2
240,Sahara occidental,ESH,Western Sahara,EH,732,24.5000,-13.0
241,Yémen,YEM,Yemen,YE,887,15.0000,48.0
242,Zambie,ZMB,Zambia,ZM,894,-15.0000,30.0


Supprimons maintenant la colonne Country.

In [4558]:
coordinates_df = coordinates_df.drop(columns=[features[0]])

coordinates_df.head()

,Pays,Code ISO3,Alpha-2 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AFG,AF,4,33.0000,65.0
1,Albanie,ALB,AL,8,41.0000,20.0
2,Algérie,DZA,DZ,12,28.0000,3.0
3,Samoa américaines,ASM,AS,16,-14.3333,-170.0
4,Andorre,AND,AD,20,42.5000,1.6


On recupère la latitude et la longitude de la France

In [4559]:
france_lat = coordinates_df[coordinates_df['Pays'] == 'France']['Latitude (average)'].sum()
france_long = coordinates_df[coordinates_df['Pays'] == 'France']['Longitude (average)'].sum()


Maintenant créons une fonction qui permet de mesurer une distance entre deux coordonnées. Appliquons cette fonction à nos données

In [4560]:
def distance_gps(table,ref_lon,lon_name,ref_lat,lat_name):
    R = 6371
    ref_lat = radians(ref_lat)
    ref_lon = radians(ref_lon)
    table[lon_name] = np.radians(table[lon_name])
    table[lat_name] = np.radians(table[lat_name])

    table['dlon'] = table[lon_name] - ref_lon
    table['dlat'] = table[lat_name] - ref_lat
    table['a'] = (np.sin(table['dlat']/2))**2 + np.cos(ref_lat) * np.cos(table[lat_name]) * (np.sin(table['dlon']/2))**2
    table['c'] = 2 * np.arctan2(np.sqrt(table.a), np.sqrt(1-table.a))
    table['Distance à la France'] = round(R * table.c,0)
    table = table.drop(columns = ['dlon','dlat','a','c'])
    return table


In [4561]:
coordinates_df = distance_gps(coordinates_df,france_long,'Longitude (average)',france_lat,'Latitude (average)')
coordinates_df.head()

,Pays,Code ISO3,Alpha-2 code,Numeric code,Latitude (average),Longitude (average),Distance à la France
0,Afghanistan,AFG,AF,4,0.575959,1.134464,5447.0
1,Albanie,ALB,AL,8,0.715585,0.349066,1550.0
2,Algérie,DZA,DZ,12,0.488692,0.052360,2003.0
3,Samoa américaines,ASM,AS,16,-0.250163,-2.967060,16415.0
4,Andorre,AND,AD,20,0.741765,0.027925,390.0


On ne conserve maintenant que les colonnes   `Pays et distance à la France`

In [4562]:
coordinates_df = coordinates_df[['Pays','Distance à la France']]
coordinates_df.head()

,Pays,Distance à la France
0,Afghanistan,5447.0
1,Albanie,1550.0
2,Algérie,2003.0
3,Samoa américaines,16415.0
4,Andorre,390.0


## 3.2 Stabilité politique
--------------------------------------

In [4563]:
political_stability_df.head()

,Year (N),ISO3,Rank N,Score N,FR_country
0,2017,NOR,1,"92,4",Norvège
1,2017,SWE,2,"91,73",Suède
2,2017,FIN,3,"91,08",Finlande
3,2017,DNK,4,"89,64",Danemark
4,2017,NLD,5,"88,72",Pays-Bas


Même problème changer le nom de la colonne ISO3 mise des même nom de pays

In [4564]:
political_stability_df = political_stability_df.rename(columns={'ISO3' : features[2]})
political_stability_df.head()

,Year (N),Code ISO3,Rank N,Score N,FR_country
0,2017,NOR,1,"92,4",Norvège
1,2017,SWE,2,"91,73",Suède
2,2017,FIN,3,"91,08",Finlande
3,2017,DNK,4,"89,64",Danemark
4,2017,NLD,5,"88,72",Pays-Bas


Changeons les noms à partir de la dénominbation FAO

In [4565]:
political_stability_df = pd.merge(fao_codes_df[['Pays',features[2]]],political_stability_df,how='right',on=features[2])
political_stability_df.head()

,Pays,Code ISO3,Year (N),Rank N,Score N,FR_country
0,Norvège,NOR,2017,1,"92,4",Norvège
1,Suède,SWE,2017,2,"91,73",Suède
2,Finlande,FIN,2017,3,"91,08",Finlande
3,Danemark,DNK,2017,4,"89,64",Danemark
4,Pays-Bas,NLD,2017,5,"88,72",Pays-Bas


Supprimons la colonne FR_country

In [4566]:
political_stability_df = political_stability_df.drop(columns=['FR_country'])
political_stability_df.head()

,Pays,Code ISO3,Year (N),Rank N,Score N
0,Norvège,NOR,2017,1,"92,4"
1,Suède,SWE,2017,2,"91,73"
2,Finlande,FIN,2017,3,"91,08"
3,Danemark,DNK,2017,4,"89,64"
4,Pays-Bas,NLD,2017,5,"88,72"


On ne garde que les colonne `Score N` et `Pays` <br>
Et on renomme Score N en stabilité politique

In [4567]:
political_stability_df = political_stability_df[['Pays','Score N']]
political_stability_df = political_stability_df.rename(columns={'Score N': 'Stabilité politique'})
political_stability_df.head()

,Pays,Stabilité politique
0,Norvège,"92,4"
1,Suède,"91,73"
2,Finlande,"91,08"
3,Danemark,"89,64"
4,Pays-Bas,"88,72"


In [4568]:
political_stability_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Pays                 177 non-null    object
 1   Stabilité politique  180 non-null    object
dtypes: object(2)
memory usage: 4.2+ KB


Le score de stabilité politique est en float ceci est du au séparateur décimal du nombre, une virgule.<br>
Nous allons corrigé et le passer en format float.

In [4569]:
political_stability_df['Stabilité politique'] = political_stability_df['Stabilité politique'].str.replace(',','.')
political_stability_df['Stabilité politique'] = pd.to_numeric(political_stability_df['Stabilité politique'])

In [4570]:
political_stability_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Pays                 177 non-null    object 
 1   Stabilité politique  180 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.2+ KB


In [4571]:
display(political_stability_df[political_stability_df.Pays.isna()])

,Pays,Stabilité politique
37,NaN,77.90
74,NaN,70.12
81,NaN,69.55


Les Pays vide correspondent dans l'ordre à l'OCDE, Chypre Nord, et le Kosovo qui ne sont pas recencé dans les pays de la FAO (OCDE n'est pas un pays).
On ne garde que les valeurs non-nulle

In [4572]:
political_stability_df = political_stability_df[political_stability_df.Pays.notna()]

## 3.3 Langue la plus parlée
----------------------------------------

In [4573]:
languages_df.head()

,Country,Most used language,ISO3
0,Afghanistan,Pashto,AFG
1,Albania,Albanian,ALB
2,Algeria,Arabic,DZA
3,Angola,Portuguese,AGO
4,Argentina,Spanish,ARG


In [4574]:
languages_df = languages_df.rename(columns={'ISO3' : features[2]})
languages_df.head()

,Country,Most used language,Code ISO3
0,Afghanistan,Pashto,AFG
1,Albania,Albanian,ALB
2,Algeria,Arabic,DZA
3,Angola,Portuguese,AGO
4,Argentina,Spanish,ARG


In [4575]:
languages_df = pd.merge(fao_codes_df[['Pays',features[2]]],languages_df,how='right',on=features[2])
languages_df.head()

,Pays,Code ISO3,Country,Most used language
0,Afghanistan,AFG,Afghanistan,Pashto
1,Albanie,ALB,Albania,Albanian
2,Algérie,DZA,Algeria,Arabic
3,Angola,AGO,Angola,Portuguese
4,Argentine,ARG,Argentina,Spanish


Supprimons les colonnes country et Code ISO3
On renomme Most used language en langue courante

In [4576]:
languages_df = languages_df.drop(columns=['Country',features[2]])
languages_df = languages_df.rename(columns = {'Most used language' : 'langue courante'})
languages_df.head()

,Pays,langue courante
0,Afghanistan,Pashto
1,Albanie,Albanian
2,Algérie,Arabic
3,Angola,Portuguese
4,Argentine,Spanish


## 3.4 Disponibilité alimentaire
----------------------------------------

In [4577]:
features = food_availability_df.columns

food_availability_df.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


Dans le cadre de notre étude nous ne conserverons que la viande de volaille

In [4578]:
food_availability_df = food_availability_df[food_availability_df['Produit'].str.contains('Volailles',na= False)]
food_availability_df.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
651,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2734,Viande de Volailles,2017,2017,Milliers de tonnes,28.0,S,Données standardisées
652,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2017,2017,Milliers de tonnes,29.0,S,Données standardisées
653,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2734,Viande de Volailles,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
654,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2017,2017,Milliers de tonnes,57.0,S,Données standardisées
655,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5123,Pertes,2734,Viande de Volailles,2017,2017,Milliers de tonnes,2.0,S,Données standardisées


Nous allons maintenant garder uniquement les colonnes utiles soit : <br>
<li>Le pays</li>
<li>Les Élements</li>
<li>Les produits</li>
<li>L'année</li>
<li>Les valeurs</li>

In [4579]:
food_availability_df = pd.DataFrame({'Pays' : food_availability_df.Zone, 'Élément' : food_availability_df['Élément'],'Produit' : food_availability_df['Produit'], 'Année' : food_availability_df['Année'], 'Valeur' : food_availability_df['Valeur']})

display(food_availability_df['Élément'].drop_duplicates().to_list())

['Production',
 'Importations - Quantité',
 'Variation de stock',
 'Disponibilité intérieure',
 'Pertes',
 'Résidus',
 'Nourriture',
 'Disponibilité alimentaire en quantité (kg/personne/an)',
 'Disponibilité alimentaire (Kcal/personne/jour)',
 'Disponibilité de protéines en quantité (g/personne/jour)',
 'Disponibilité de matière grasse en quantité (g/personne/jour)',
 'Exportations - Quantité',
 'Alimentation pour touristes',
 'Traitement',
 'Autres utilisations (non alimentaire)',
 'Aliments pour animaux',
 'Semences']

On transpose la table de disponibilité alimentaire en ne conservant que les variables souhaitées : `Importations`, `Disponibilité intérieure`, `Disponibilité de protéines en quantité (g/personne/jour)`

In [4580]:
usefull_element = ['Importations - Quantité','Disponibilité intérieure']


init_food_df = food_availability_df[food_availability_df['Élément'] == 'Production']
init_food_df = init_food_df.drop(columns=['Élément'])
init_food_df = init_food_df.rename(columns={'Valeur' : 'Production'})

def partition(element_tab,init_df,final_df):
    for x in element_tab:
        inter_df = init_df[init_df['Élément'] == x]
        inter_df = inter_df.drop(columns = ['Élément'])
        inter_df = inter_df.rename(columns = {'Valeur' : x})
        final_df = pd.merge(final_df,inter_df,how='outer',on=['Pays','Produit','Année'])
    return final_df

food_availability_df = partition(usefull_element,food_availability_df,init_food_df)


display(food_availability_df)

,Pays,Produit,Année,Production,Importations - Quantité,Disponibilité intérieure
0,Afghanistan,Viande de Volailles,2017,28.0,29.0,57.0
1,Afrique du Sud,Viande de Volailles,2017,1667.0,514.0,2118.0
2,Albanie,Viande de Volailles,2017,13.0,38.0,47.0
3,Algérie,Viande de Volailles,2017,275.0,2.0,277.0
4,Allemagne,Viande de Volailles,2017,1514.0,842.0,1739.0
...,...,...,...,...,...,...
165,Yémen,Viande de Volailles,2017,168.0,78.0,246.0
166,Zambie,Viande de Volailles,2017,49.0,12.0,60.0
167,Zimbabwe,Viande de Volailles,2017,69.0,6.0,76.0
168,Djibouti,Viande de Volailles,2017,NaN,3.0,3.0


On supprime les colonnes Produit et annnée

In [4581]:
food_availability_df = food_availability_df.drop(columns=['Produit', 'Année'])

Maintenant nous pouvons ajouter la colonne Taux d'importation qui est égal à l'importation sur la disponibilité intérieur

In [4582]:
food_availability_df['Taux importation'] = food_availability_df['Importations - Quantité']/food_availability_df['Disponibilité intérieure']

In [4583]:
food_availability_df = food_availability_df.drop(columns=['Importations - Quantité','Disponibilité intérieure'])

# 3.5 Population
-----------------------------------

In [4584]:
population_df.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


On ne conserve que le pays, la valeur de population et l'année 

In [4585]:
population_df = pd.DataFrame({'Pays' : population_df.Zone, 'Année' : population_df['Année'],'Population' : population_df.Valeur *1000})

On ne conserve l'année 2017 et on supprime la colonne année

In [4586]:
population_df = population_df[population_df['Année']== 2017]
population_df = population_df.drop(columns=['Année'])
population_df.head()

,Pays,Population
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0


## 3.6 Table Économique
--------------------------------------

In [4587]:
eco_df.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,4,Afghanistan,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,1.889635e+04,X,Ciffre de sources internationales,NaN
1,MK,Indicateurs macro,710,Afrique du Sud,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,3.490067e+05,X,Ciffre de sources internationales,NaN
2,MK,Indicateurs macro,8,Albanie,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,1.301973e+04,X,Ciffre de sources internationales,NaN
3,MK,Indicateurs macro,12,Algérie,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,1.700970e+05,X,Ciffre de sources internationales,NaN
4,MK,Indicateurs macro,276,Allemagne,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,3.690849e+06,X,Ciffre de sources internationales,NaN


On ne conserve que le pays et la colonne Valeur qui correspond au produit intérieur brut en dollars

In [4588]:
eco_df = pd.DataFrame({'Pays' : eco_df.Zone, 'Produit intérieur brut en dollars' : eco_df.Valeur *pow(10,6)})

eco_df.head()


,Pays,Produit intérieur brut en dollars
0,Afghanistan,1.889635e+10
1,Afrique du Sud,3.490067e+11
2,Albanie,1.301973e+10
3,Algérie,1.700970e+11
4,Allemagne,3.690849e+12


On couple les table de popuylation et économique

In [4589]:
eco_df = pd.merge(eco_df,population_df,how='left', on = 'Pays')

display(eco_df)

,Pays,Produit intérieur brut en dollars,Population
0,Afghanistan,1.889635e+10,36296113.0
1,Afrique du Sud,3.490067e+11,57009756.0
2,Albanie,1.301973e+10,2884169.0
3,Algérie,1.700970e+11,41389189.0
4,Allemagne,3.690849e+12,82658409.0
...,...,...,...
207,Viet Nam,2.237799e+11,94600648.0
208,Yémen,2.073938e+10,27834819.0
209,Zambie,2.586816e+10,16853599.0
210,Zimbabwe,2.204090e+10,14236595.0


On créé la colonne PIB/habitant et supprime les colonne précédentes

In [4590]:
eco_df['PIB/habitant'] = eco_df['Produit intérieur brut en dollars']/eco_df['Population']
eco_df = eco_df.drop(columns=['Population','Produit intérieur brut en dollars'])
eco_df.head()

,Pays,PIB/habitant
0,Afghanistan,520.616409
1,Afrique du Sud,6121.876572
2,Albanie,4514.204908
3,Algérie,4109.696001
4,Allemagne,44651.829102


# 4. Fusion des tables précédemment créé
---------------------------------------

On s'intéresse aux dimensions des tables initiales

In [4591]:
coordinates_df.shape

(244, 2)

In [4592]:
political_stability_df.shape

(177, 2)

In [4593]:
languages_df.shape

(200, 2)

In [4594]:
food_availability_df.shape

(170, 3)

In [4595]:
eco_df.shape

(212, 2)

On lie la table des distance (coordonnées) avec la table de nourriture au sein de notre table finale avec pour référentiel la table de nourriture.

In [4596]:
final_df = pd.merge(food_availability_df,coordinates_df,how='left',on='Pays')
final_df.shape


(170, 4)

On lie ensuite la table finale avec la table de stabilité politique.

In [4597]:
final_df = pd.merge(final_df,political_stability_df,how='left',on='Pays')
final_df.shape

(170, 5)

Nous lions la table finale avec la table des langues.

In [4598]:
final_df = pd.merge(final_df,eco_df,how='left',on='Pays')
final_df.shape

(170, 6)


On lie enfin la table économique avec la table de finale

In [4599]:
final_df = pd.merge(final_df,languages_df,how='left',on='Pays')
final_df.shape

(170, 7)

# 5. Traitement des données manquants
-----------------------------------------------

In [4600]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pays                  170 non-null    object 
 1   Production            168 non-null    float64
 2   Taux importation      170 non-null    float64
 3   Distance à la France  170 non-null    float64
 4   Stabilité politique   155 non-null    float64
 5   PIB/habitant          170 non-null    float64
 6   langue courante       170 non-null    object 
dtypes: float64(5), object(2)
memory usage: 10.6+ KB


Les valeurs manquantes dans la colonne stabilité politique sont remplacées par la moyenne de la stabilité politique mondiale

On ne peut appliquer de formule sur des données qualitative on exclu

In [4601]:
final_df['Stabilité politique'] = final_df['Stabilité politique'].replace(np.nan,final_df['Stabilité politique'].mean())



In [4602]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pays                  170 non-null    object 
 1   Production            168 non-null    float64
 2   Taux importation      170 non-null    float64
 3   Distance à la France  170 non-null    float64
 4   Stabilité politique   170 non-null    float64
 5   PIB/habitant          170 non-null    float64
 6   langue courante       170 non-null    object 
dtypes: float64(5), object(2)
memory usage: 10.6+ KB


In [4603]:
display(final_df[final_df['Taux importation'].isna()])

,Pays,Production,Taux importation,Distance à la France,Stabilité politique,PIB/habitant,langue courante


Nous prenons la moyenne de notre échantillon pour les taux d'imporation manquants

In [4604]:
final_df['Taux importation'] = final_df['Taux importation'].replace(np.nan,final_df['Taux importation'].mean())

In [4605]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pays                  170 non-null    object 
 1   Production            168 non-null    float64
 2   Taux importation      170 non-null    float64
 3   Distance à la France  170 non-null    float64
 4   Stabilité politique   170 non-null    float64
 5   PIB/habitant          170 non-null    float64
 6   langue courante       170 non-null    object 
dtypes: float64(5), object(2)
memory usage: 10.6+ KB


In [4606]:
final_df['Production'] = final_df['Production'].replace(np.nan,final_df['Production'].mean())

On exclut la France de l'analyse

In [4607]:
final_df = final_df[final_df['Pays']!= 'France']

Toutes les données sont maintenant remplies

# 6. Enregistrement de notre fichier en format csv
----------------------------------------

In [4608]:
final_df.to_csv(PATH+'final_df.csv',index=False)